# Intelligent Element Design

The idea of the intelligent element is to automate the creation of Keras models and data generation for complex nested structures.

In [1]:
from keras import Model
from keras import layers as L

import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2
    
from IntelligentElement import IntelligentElement, get_children_sum_last_output_shapes

Using TensorFlow backend.


# Sample synthesized use

In [2]:

regions_id = {'countryside' : 0, 'central' : 1, 'innerside' : 2}
def genMonthDetails():
    ndays = np.random.randint(4,7)
    ans=[]
    for i in range(ndays):
        ans.append({'day' : i, 'raw_amount' : np.random.randint(400,7000)})
    return ans
    
def genClientData():
    regions = ['countryside', 'central', 'innerside']
    
    ans = {'income' : np.random.randint(10,50), 'age' : np.random.randint(20,70), 
           'residence_region' : regions[np.random.randint(len(regions))] }
    
    ans['last_transactions'] = []
    nlast = np.random.randint(3,10)
    for i in range(nlast):
        last_transaction = {'month' : i, 'value':np.random.randint(1,5)}
        last_transaction['details'] = genMonthDetails()
        ans['last_transactions'].append(last_transaction)
    
    ans['picture'] = np.ones((128,128,3))
    
    return ans

clientdata = [genClientData() for i in range(20)]

In [3]:
pd.DataFrame(clientdata).head()

,age,income,last_transactions,picture,residence_region
0,43,38,"[{'month': 0, 'value': 4, 'details': [{'day': ...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",central
1,58,38,"[{'month': 0, 'value': 1, 'details': [{'day': ...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",countryside
2,32,12,"[{'month': 0, 'value': 3, 'details': [{'day': ...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",countryside
3,21,21,"[{'month': 0, 'value': 3, 'details': [{'day': ...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",innerside
4,49,43,"[{'month': 0, 'value': 2, 'details': [{'day': ...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",innerside


In [4]:
picture_data = [x['picture'] for x in clientdata]
picture_shape = (128,128,3)
inp=L.Input(picture_shape)
x=L.Conv2D(10,(3,3), activation='relu', padding='same')(inp)
x=L.Flatten()(x)
x=L.Dense(5,activation='relu')(x)
picture_model = Model(inputs=inp, outputs=x)
print('Original model')
picture_model.summary()

picture_ie = IntelligentElement(picture_data, picture_model, picture_shape, name='picture_ie')

m, ii, oo = picture_ie.retrieve_model_inputs_outputs()
print('\n\nRetrieved model')
m.summary()

Original model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 10)      280       
_________________________________________________________________
flatten_1 (Flatten)          (None, 163840)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 819205    
Total params: 819,485
Trainable params: 819,485
Non-trainable params: 0
_________________________________________________________________


Retrieved model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp_picture_ie (InputLayer)  (None, 128, 128, 3)       0         
_____________________________________

In [5]:
picture_ie.get_batch([0,4,5,8,9])[0].shape

(5, 128, 128, 3)

In [6]:
details_data = [[y['details'] for y in x['last_transactions']] for x in clientdata]
details_shape = (None, None, 3)
def details_preproc_function(x):
    #receives a list of dictionaries with keys 'day' and 'raw_amount'
    #append a 1 to indicate that these values were not padded
    ans = []
    for item in x:
        monthdetails = []
        for detaildata in item:
            monthdetails.append([detaildata['day']/30, detaildata['raw_amount']/10000, 1])
        ans.append(np.array(monthdetails))
    
    #handle case when list is empty
    if len(ans) == 0:
        ans.append(np.array([[0,0,0]]))
    return ans

inp = L.Input(details_shape)
x = L.TimeDistributed(L.LSTM(64))(inp)
x = L.TimeDistributed(L.Dense(8, activation='relu'))(x)
details_model = Model(inputs=inp, outputs=x)
print('Original model')
details_model.summary()

details_ie = IntelligentElement(details_data, details_model, details_shape, 
                                    preprocess_function=details_preproc_function, name='details_ie')

m, ii, oo = details_ie.retrieve_model_inputs_outputs()
print('\n\nRetrieved model')
m.summary()

Original model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 64)          17408     
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 8)           520       
Total params: 17,928
Trainable params: 17,928
Non-trainable params: 0
_________________________________________________________________


Retrieved model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp_details_ie (InputLayer)  (None, None, None, 3)     0         
_________________________________________________________________
m_details_ie (Model)         (None, None, 8)           17928     
Total params: 17,928
Trainable params: 

In [7]:
details_data[0]

[[{'day': 0, 'raw_amount': 1408},
  {'day': 1, 'raw_amount': 1313},
  {'day': 2, 'raw_amount': 2925},
  {'day': 3, 'raw_amount': 5007},
  {'day': 4, 'raw_amount': 2837}],
 [{'day': 0, 'raw_amount': 1663},
  {'day': 1, 'raw_amount': 4154},
  {'day': 2, 'raw_amount': 2062},
  {'day': 3, 'raw_amount': 483}],
 [{'day': 0, 'raw_amount': 3417},
  {'day': 1, 'raw_amount': 1194},
  {'day': 2, 'raw_amount': 1384},
  {'day': 3, 'raw_amount': 5378},
  {'day': 4, 'raw_amount': 876},
  {'day': 5, 'raw_amount': 2227}],
 [{'day': 0, 'raw_amount': 6829},
  {'day': 1, 'raw_amount': 1708},
  {'day': 2, 'raw_amount': 5237},
  {'day': 3, 'raw_amount': 886}],
 [{'day': 0, 'raw_amount': 2947},
  {'day': 1, 'raw_amount': 4091},
  {'day': 2, 'raw_amount': 4877},
  {'day': 3, 'raw_amount': 4598},
  {'day': 4, 'raw_amount': 4214},
  {'day': 5, 'raw_amount': 2221}]]

In [8]:
details_preproc = details_preproc_function(details_data[1])
details_preproc

[array([[0.        , 0.5173    , 1.        ],
        [0.03333333, 0.6646    , 1.        ],
        [0.06666667, 0.0529    , 1.        ],
        [0.1       , 0.4474    , 1.        ],
        [0.13333333, 0.1158    , 1.        ]]),
 array([[0.        , 0.146     , 1.        ],
        [0.03333333, 0.1188    , 1.        ],
        [0.06666667, 0.5545    , 1.        ],
        [0.1       , 0.3008    , 1.        ]]),
 array([[0.        , 0.0946    , 1.        ],
        [0.03333333, 0.3716    , 1.        ],
        [0.06666667, 0.6256    , 1.        ],
        [0.1       , 0.1334    , 1.        ],
        [0.13333333, 0.3665    , 1.        ]]),
 array([[0.        , 0.3188    , 1.        ],
        [0.03333333, 0.6686    , 1.        ],
        [0.06666667, 0.6714    , 1.        ],
        [0.1       , 0.4735    , 1.        ],
        [0.13333333, 0.5138    , 1.        ]]),
 array([[0.        , 0.0883    , 1.        ],
        [0.03333333, 0.6624    , 1.        ],
        [0.06666667, 0.046

In [9]:
details_ie.model.output_shape[-1]

8

In [10]:
b = details_ie.get_batch([0,1,2,8,9])

In [11]:
b[0][1]

array([[[0.        , 0.5173    , 1.        ],
        [0.03333333, 0.6646    , 1.        ],
        [0.06666667, 0.0529    , 1.        ],
        [0.1       , 0.4474    , 1.        ],
        [0.13333333, 0.1158    , 1.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.146     , 1.        ],
        [0.03333333, 0.1188    , 1.        ],
        [0.06666667, 0.5545    , 1.        ],
        [0.1       , 0.3008    , 1.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.0946    , 1.        ],
        [0.03333333, 0.3716    , 1.        ],
        [0.06666667, 0.6256    , 1.        ],
        [0.1       , 0.1334    , 1.        ],
        [0.13333333, 0.3665    , 1.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.3188    , 1.        ],
        [0.03333333, 0.6686    , 1.        ],
        [0.06666667, 0.6714    , 1.        ],
        [0.1       , 0.4735 

In [15]:
transaction_data = [x['last_transactions'] for x in clientdata]
transaction_shape = (None, 3)

def transaction_preproc_function(x):
    #receives a dictionary with keys 'month' and 'value'
    #append a 1 to indicate that these values were not padded
    ans = []
    for item in x:
        ans.append([item['month']/12, item['value']/1000, 1])
    
    #handle case when list is empty
    if len(ans) == 0:
        ans.append([0,0,0])
    
    return np.array(ans)

inp = L.Input( (None,3+details_ie.model.output_shape[-1]) )

#inp = L.Input(transaction_shape)
x = L.LSTM(64)(inp)
x = L.Dense(16, activation='relu')(x)
x = L.Dense(8, activation='relu')(x)
transaction_model = Model(inputs=inp, outputs=x)
print('Original model')
transaction_model.summary()

transaction_ie = IntelligentElement(transaction_data, transaction_model, transaction_shape, children_ie=details_ie,
                                    preprocess_function=transaction_preproc_function, name='transaction_ie')

m, ii, oo = transaction_ie.retrieve_model_inputs_outputs()
print('\n\nRetrieved model')
m.summary()

Original model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, 11)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                19456     
_________________________________________________________________
dense_5 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 136       
Total params: 20,632
Trainable params: 20,632
Non-trainable params: 0
_________________________________________________________________


Retrieved model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp_details_ie (InputLayer)     (None, 

In [16]:
transaction_ie.preprocess_function(transaction_ie.data[3])

array([[0.        , 0.003     , 1.        ],
       [0.08333333, 0.004     , 1.        ],
       [0.16666667, 0.004     , 1.        ],
       [0.25      , 0.001     , 1.        ]])

In [17]:
bb = transaction_ie.get_batch([0,4,2,3,9])
print(len(bb)) #2 because there is input for the nested details element
bb[1][3]

2


array([[0.        , 0.003     , 1.        ],
       [0.08333333, 0.004     , 1.        ],
       [0.16666667, 0.004     , 1.        ],
       [0.25      , 0.001     , 1.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [22]:
#numerical for age and income, one-hot encoding for residence_region
client_data = [[x['age'], x['income'], x['residence_region']] for x in clientdata]
client_shape = (5,)
def client_preproc_function(x):
    #receives a list with age, income and residence_region
    idx = regions_id[x[2]]
    region_1hot = [0] * len(regions_id)
    region_1hot[idx] = 1

    return np.array([x[0]/100, np.log(1+x[1])]+region_1hot)
dim = get_children_sum_last_output_shapes([picture_ie,transaction_ie]) + client_shape[0]
print( dim )
inp=L.Input(  (dim,) )
            
x=inp
x=L.Dense(8, activation='relu')(x)
x=L.Dense(8, activation='relu')(x)
client_model = Model(inputs=inp, outputs=x)

print('Original model')
client_model.summary()

client_ie = IntelligentElement(client_data, client_model, client_shape, preprocess_function=client_preproc_function,
                               children_ie=[picture_ie, transaction_ie], name='client_ie')

m, ii, oo = client_ie.retrieve_model_inputs_outputs()
print('\n\nRetrieved model')
m.summary()

18
Original model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 18)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 152       
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 72        
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________


AssertionError: children_ie must contain only IntelligentElement

In [20]:
client_ie.preprocess_function(client_ie.data[3])

array([0.47      , 2.89037176, 0.        , 0.        , 1.        ])

In [21]:
b = client_ie.get_batch([0,4,2,8,9])
print(b[0])
print(b[2][3])

[[[[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  ...

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]]


 [[[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  ...

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1